# Analisando o bot

Este jupyter-notebook vai auxiliar na análise de um RASA bot.

## Instalação

### Configurando jupyter

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

### Instalação do Rasa


Verificando versões:

In [2]:
import rasa_nlu
import spacy

print("rasa_nlu: {}".format(rasa_nlu.__version__))
print("Loading spaCy language model...")
print(spacy.load("pt")("Olá Mundo!"))

rasa_nlu: 0.12.3
Loading spaCy language model...
Olá Mundo!


### Treinando o modelo do Rasa NLU

Na celula abaixo todas as intents na pasta `../data/intents/` serão utilizadas para gerar os gráficos de avaliação do bot.

Ela irá treinar o modelo e executar a avaliação que pode ser verificada na saída da célula abaixo.

In [3]:
import os

from rasa_nlu.training_data import load_data
from rasa_nlu.model import Trainer
from rasa_nlu import config


intents_directory = '../../bot/data/intents/'

intents = {}

for intent_file in os.listdir(intents_directory):
    intent_file_path = os.path.join(intents_directory, intent_file)

    intents[intent_file] = {}

    intent_list = []
    intent_name = None

    with open(intent_file_path) as f:
        lines = f.readlines()

        for line in lines:
            line = line.strip()

            if line.startswith('##'):
                if intent_name is not None:
                    intents[intent_file][intent_name] = intent_list
                intent_name = line.replace('## intent:', '') 
                intent_list = []

            elif line.startswith('- '):
                intent_list.append(line.replace('- ', ''))


# loading the nlu training samples
training_data = load_data(intents_directory)

# trainer to educate our pipeline
trainer = Trainer(config.load("../../bot/nlu_config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

from rasa_nlu.evaluate import run_evaluation
run_evaluation('../../bot/data/intents/', model_directory)

FileNotFoundError: [Errno 2] No such file or directory: '../bot/data/intents/'

### Avaliação do modelo de NLU treinado

`Caso queira testar mensagens específicas mande a mensagem na célula seguinte`

Quando quiser parar o teste diga **pare**

In [5]:
pprint(interpreter.parse('posso terceirizar a captação de recursos?'))

{
  "intent": {
    "name": "curiosidades_captacao_de_recursos",
    "confidence": 0.9602228403091431
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "curiosidades_captacao_de_recursos",
      "confidence": 0.9602228403091431
    },
    {
      "name": "afirmar_curiosidades",
      "confidence": 0.09013605117797852
    },
    {
      "name": "curiosidades_deducao_imposto_de_renda_incentivador",
      "confidence": 0.052063263952732086
    },
    {
      "name": "processo_periodo",
      "confidence": 0.0518600270152092
    },
    {
      "name": "curiosidades_valor_maximo_geral",
      "confidence": 0.04678261652588844
    },
    {
      "name": "negar",
      "confidence": 0.041245631873607635
    },
    {
      "name": "curiosidades_elegibilidade_de_empresa_para_beneficios",
      "confidence": 0.030568117275834084
    },
    {
      "name": "curiosidades_valor_maximo_pessoa_fisica",
      "confidence": 0.020352115854620934
    },
    {
      "name": "afirmar",
     